# Qualitative Data & Charge Analysis

This notebook focuses on the qualitative aspects of the data, including:
1.  **Data Loading & Filtering**: Geometrical cuts and basic stats.
2.  **Charge Analysis**: Distribution of charges, hits, and time.
3.  **Detailed Single Event Analysis**: 3D visualization, vertex reconstruction checks, and **Incidence Angle Calculations**.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Add project root to path
sys.path.append(os.path.abspath('..')) 

from data_processing.utils import get_single_event_df
from visualization.plots import distribution

# Set plotting style
sns.set_theme(style="whitegrid")

## 1. Data Loading & Filtering
Load the main event dataframe.

In [ ]:
# Paths
DF_PKL_PATH = r"../base_de_donnee/df.pkl"
DF_EVENTS_PKL_PATH = r"../base_de_donnee/df_events/df.pkl"

df = None

for pkl_path in [DF_PKL_PATH, DF_EVENTS_PKL_PATH]:
    if os.path.exists(pkl_path):
        print(f"Loading data from {pkl_path}...")
        try:
            df = pd.read_pickle(pkl_path)
            print(f"Successfully loaded {len(df)} events.")
            break
        except Exception as e:
            print(f"Failed to load pickle {pkl_path}: {e}")

if df is None:
     print("Warning: No data loaded. Creating dummy data for demonstration.")
     df = pd.DataFrame({
        'energy': np.random.exponential(1000, 1000),
        'charge_totale': np.random.exponential(5000, 1000),
        'n_hits': np.random.poisson(500, 1000),
        'dwall': np.random.uniform(0, 1000, 1000),
        'particleStop_x': np.random.uniform(-4000, 4000, 1000),
        'particleStop_y': np.random.uniform(-4000, 4000, 1000),
        'particleStop_z': np.random.uniform(-4000, 4000, 1000),
        'max_charge': np.random.uniform(10, 100, 1000)
    })

In [ ]:
# Geometrical Filtering
if 'particleStop_x' in df.columns:
    print("Applying geometrical filtering (particleStop)...\n")
    limites_x, limites_y, limites_z = 3242.766113, 3242.766113, 3296.471191
    mask = (abs(df["particleStop_x"]) > limites_x) | (abs(df["particleStop_y"]) > limites_y) | (abs(df["particleStop_z"]) > limites_z)
    df = df[~mask].copy()
    print(f"Events remaining: {len(df)}")

## 2. Descriptive Analysis

In [ ]:
cols_to_desc = ["charge_totale", "n_hits", "energy", "dwall"]
display(df[[c for c in cols_to_desc if c in df.columns]].describe())

## 3. Deep Dive: Single Event with Geometry & Angles
Here we extract a single event and calculate the specific incidence angles as requested.

In [ ]:
SINGLE_EVENT_PKL = r"../base_de_donnee/df_single_event.pkl"
single_event = None

if os.path.exists(SINGLE_EVENT_PKL):
    single_event = pd.read_pickle(SINGLE_EVENT_PKL)
    print("Loaded single event from pickle.")
elif df is not None and 'max_charge' in df.columns:
     # Find event with max charge close to target
    target = 800
    idx = (df['max_charge'] - target).abs().idxmin()
    print(f"Extracting event {idx}...")
    single_event = get_single_event_df(df, idx)

In [ ]:
HK = {
    'height': 6575.1,
    'cylinder_radius': 6480 / 2,
    'PMT_radius': 25.4,
    'maxi': 3296.4712
}

def calculate_angle_incidence(pmt_pos, vertex_pos, hk_params):
    x0, y0, z0 = pmt_pos
    vx, vy, vz = vertex_pos
    maxi = hk_params['maxi']
    
    # 1. Vector: Vertex -> PMT (Photon direction)
    v = np.array([x0 - vx, y0 - vy, z0 - vz])
    v_norm = np.linalg.norm(v)
    
    # 2. Normal Vector at PMT position
    if abs(z0) > (maxi - 10): # Top/Bottom caps
        n = np.array([0, 0, 1 if z0 > 0 else -1]) 
    else: # Cylinder side
        n = np.array([x0, y0, 0])
    
    n_norm = np.linalg.norm(n)
    
    # 3. Angle (Radians -> Degrees)
    dot_prod = np.dot(v, n)
    theta_deg = np.degrees(np.arccos(dot_prod / (v_norm * n_norm)))
    
    return theta_deg

if single_event is not None:
    print("Calculating angles...")
    # Assuming 'vertex_x' etc are constant
    vertex = single_event.iloc[0][['vertex_x', 'vertex_y', 'vertex_z']].values
    
    single_event['angle_incidence'] = single_event.apply(
        lambda row: calculate_angle_incidence(
            [row['hitx'], row['hity'], row['hitz']], 
            vertex, 
            HK
        ), axis=1
    )
    print("Angles calculated.")

In [ ]:
if single_event is not None:
    # 3D Plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(single_event['hitx'], single_event['hity'], single_event['hitz'], 
                    c=single_event['charge'], cmap='viridis', s=20)
    ax.scatter(vertex[0], vertex[1], vertex[2], c='red', s=100, marker='*', label='Vertex')
    plt.colorbar(sc, label='Charge')
    ax.set_title("3D Event Display & Vertex")
    plt.show()

    # Angle Hist
    plt.figure(figsize=(8, 5))
    sns.histplot(single_event['angle_incidence'], bins=30, kde=True)
    plt.title("Incidence Angle Distribution")
    plt.xlabel("Angle (Deg)")
    plt.show()